<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/%EC%97%AC%EC%A7%84%EC%98%81/%EC%82%BC%EC%84%B1%EC%8B%A4%EC%8A%B54%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisite

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Share/Samsung_share/

/content/drive/MyDrive/Share/Samsung_share


In [ ]:
ls

imdb.csv  삼성실습4일차.ipynb


In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_openai
!pip install langchain_community
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import openai
import asyncio
import os
import json
from functools import partial
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_community.callbacks import get_openai_callback
from langchain_community.chat_models import ChatOpenAI
import pprint
pp = pprint.PrettyPrinter(indent=4)

# In-context learning

In [ ]:
api_model_name = "/root/vllm_api/Meta-Llama-3-8B"

llm = OpenAI(
            model_name=api_model_name,
            openai_api_base="http://147.47.208.84:9999/v1",
            openai_api_key="EMPTY",
            max_retries=3,
            temperature=0.7,
            max_tokens=500,
        )

## Playground (w/ pre-trained model)

In [ ]:
response = llm.invoke("Hello World!")
print(response)

 This is my first post here. I'm hoping to get some good advice from the members here. I'm hoping to start up a small business eventually, but for now I'm just trying to figure out what I can do to earn some extra money. I was thinking of starting a blog, and selling some of the keys I use for my business. I have a lot of them, and I'm not sure how many people would actually buy them from me. I'm hoping that I would be able to make some good money from this business, but I'm not sure if that would be possible. I'm hoping that you guys could give me some good advice on this subject.
Hi there and welcome to the forum!
Where are the keys you are trying to sell from?


In [ ]:
input_text = "Hello!"
print(llm.invoke(input_text))

NameError: name 'llm' is not defined

## Lets get into ICL

In [ ]:
zero_shot_example = """Translate English to Korean:
train => """
print(llm.invoke(zero_shot_example, stop=['\n']))

 전철


In [ ]:
zero_shot_example2 = """train => """
print(llm.invoke(zero_shot_example2, stop=['\n']))

0.84027


In [ ]:
one_shot_example = """apple => 사과
train =>"""
print(llm.invoke(one_shot_example, stop=['\n']))

 열차


In [ ]:
three_shot_example = """Translate English to Korean:
apple => 사과
ruler => 자
train =>"""
print(llm.invoke(three_shot_example, stop=['\n']))

 기차


## Sentiment classification w/ ICL

### Setting

In [ ]:
from datasets import load_dataset
import random

dataset_test = load_dataset('imdb',split='test')
df=pd.DataFrame(dataset_test)
test_df = df.sample(n=100)

In [ ]:
print(f"data length: {len(test_df)}")
test_df.head()

data length: 100


,text,label
13420,"From the start of ""The Edge Of Love"", the view...",1
16650,Wow. this movie is the voice of a climbing gen...,1
20659,"A good, but not great film, ""The Great Dictato...",1
17282,The world is going to miss John Frankenheimer....,1
20854,Following on directly from the last episode of...,1


In [ ]:
# check the first data
print(test_df.iloc[0]['text'])

From the start of "The Edge Of Love", the viewer is transported to the striking world of WW2 London. We follow the lives of four people who might have been created just for this movie, an exploration of female friendship and the strains caused on it by marriage and infidelity. Except one of the characters is named Dylan Thomas, perhaps the greatest English poet of the 20th century. And his reactions to the world around him were not only selfish, but at times truly despicable. <br /><br />This movie is based on Thomas' writings about love and romance. These were adapted with a sharp screenplay by Sharman MacDonald (Keira Knightley's mother). The director, John Maybury, does claim that the three other lead characters were actual people. <br /><br />All four are performed very strongly. Sienna Miller is Dylan's wife, Keira Knightley is the cabaret singer Vera Phillips. Matthew Rhys is Dylan Thomas, and Cillian Murphy is William Killick. The first section of the movie takes place in London

### Preprocess

In [ ]:
LABEL2LANG = {
    0: "negative",
    1: "positive"
}
LANG2LABEL = {
    "negative": 0,
    "positive": 1
}
# List of dictionary
data = []

for index, row in test_df.iterrows():
    data.append({
        "idx" : index,
        "text": row['text'],
        "label": row['label'],
        "label_lang": LABEL2LANG[row['label']]
    })

### Prompt setting

In [ ]:
prompt_text1 = """
This movie is awesome! -> positive
This movie is terrible! -> negative
{text} ->"""


prompt_text2 = """
[Example 1]
text: This movie is awesome!
label: positive
[Example 2]
text: This movie is terrible!
label: negative
[Example 3]
text: {text}
label:"""

## TODO ##
prompt_text3 = """
[Example 1]
text: This movie is awesome!
label: positive
[Example 2]
text: This movie is terrible!
label: negative
[Example 3]
text: {text}
label:"""

prompt_template = PromptTemplate(
    input_variables=["text"],
    template=prompt_text1
)

### Test

In [ ]:
from tqdm import tqdm
from copy import deepcopy

def test_prompt(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(text=d['text'])
      response = llm.invoke(input_text, stop=['\n'])
      tmp['response'] = response
      cleaned_response = response.strip().lower()
      tmp['cleaned_response'] = cleaned_response
      try:
          answer = LANG2LABEL[cleaned_response]
      except:
          print(f"\nidx: {d['idx']}, Error: {cleaned_response}")
          answer = -1
      tmp['answer'] = answer
      answer_list.append(tmp)
  return answer_list

def get_accuracy(data):
  correct = 0
  print("calculating accuracy")
  for d in tqdm(data):
      if d['answer'] == d['label']:
          correct += 1
  return correct / len(data)


In [ ]:
answer_list = test_prompt(prompt_template, data)
print(f"\nAccuracy: {get_accuracy(answer_list)}")

 40%|████      | 40/100 [01:04<03:55,  3.93s/it]


Error: neutral


 62%|██████▏   | 62/100 [01:10<00:10,  3.52it/s]


Error: neutral


 72%|███████▏  | 72/100 [02:05<00:32,  1.17s/it]

...


Error: neutral


100%|██████████| 100/100 [03:06<00:00,  1.87s/it]


calculating accuracy


100%|██████████| 100/100 [00:00<00:00, 343795.41it/s]

Accuracy: 0.89


In [ ]:
pp.pprint(answer_list[0])

### Switch Positive & Negative

In [ ]:
SWITCHEDLANG2LABEL = {
    "negative": 1,
    "positive": 0
}

def test_opposite_prompt(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(text=d['text'])
      response = llm.invoke(input_text, stop=['\n'])
      tmp['response'] = response
      cleaned_response = response.strip().lower()
      tmp['cleaned_response'] = cleaned_response
      try:
          answer = SWITCHEDLANG2LABEL[cleaned_response]
      except:
          print(f"\nError: {cleaned_response}")
          answer = -1
      tmp['answer'] = answer
      answer_list.append(tmp)
  return answer_list

In [ ]:
opposite_prompt_text1 = """
[Example 1]
text: This movie is awesome!
label: negative
[Example 2]
text: This movie is terrible!
label: positive
[Example 3]
text: {text}
label:"""

## TODO ##
opposite_prompt_text2 = """
[Example 1]
text: This movie is awesome!
label: negative
[Example 2]
text: This movie is terrible!
label: positive
[Example 3]
text: {text}
label:"""

opposite_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=opposite_prompt_text1
)

In [ ]:
opposite_answer_list = test_opposite_prompt(opposite_prompt_template, data)
print(f"Accuracy: {get_accuracy(opposite_answer_list)}")

100%|██████████| 100/100 [02:52<00:00,  1.72s/it]


calculating accuracy


100%|██████████| 100/100 [00:00<00:00, 469161.52it/s]

Accuracy: 0.34


### Symbolic in-context learning

In [ ]:
SYMBOLICLANG2LABEL = {
    "bar": 0,
    "foo": 1
}

def test_symbolic_prompt(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(text=d['text'])
      response = llm.invoke(input_text, stop=['\n'])
      tmp['response'] = response
      cleaned_response = response.strip().lower()
      tmp['cleaned_response'] = cleaned_response
      try:
          answer = SYMBOLICLANG2LABEL[cleaned_response]
      except:
          print(f"\nError: {cleaned_response}")
          answer = -1
      tmp['answer'] = answer
      answer_list.append(tmp)
  return answer_list

In [ ]:
symbolic_prompt_text = """
"""

symbolic_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=symbolic_prompt_text
)

In [ ]:
symbolic_answer_list = test_symbolic_prompt(symbolic_prompt_template, data)
print(f"Accuracy: {get_accuracy(symbolic_answer_list)}")

# Chain-of-Thought

In [ ]:
api_model_name = "/root/vllm_api/Meta-Llama-3-8B"

llm = OpenAI(
            model_name=api_model_name,
            openai_api_base="http://147.47.208.84:9999/v1",
            openai_api_key="EMPTY",
            max_retries=3,
            temperature=0.7,
            max_tokens=500,
        )

## Arithmetic task

* Compare direct & CoT

In [ ]:
from datasets import load_dataset
import random
gsm8k_main = load_dataset("gsm8k", "main")['test']
print(f"total length of gsm8k: {len(gsm8k_main)}")
sample_indices = random.sample([i for i in range(len(gsm8k_main))], 100)
gsm8k = []
for d in sample_indices:
    gsm8k.append(gsm8k_main[d])
print(f"total length of sampled data of gsm8k: {len(gsm8k)}")

total length of gsm8k: 1319
total length of sampled data of gsm8k: 100


In [ ]:
## Example
print(gsm8k[0]['question'])
print("-"*10)
print(gsm8k[0]['answer'])

Each pole on a road intersection has 4 street lights. If the number of poles at each intersection is 6, and the road has 4 intersections, calculate the total number of functioning street lights if 20 streetlights from the total number are not working.
----------
If the number of poles at each intersection is 6, and the road has 4 intersections, the total number of poles having street lights is 6*4=<<6*4=24>>24
Since each pole on a road intersection has 4 street lights, the total number of streetlights in all the poles is 24*4=<<24*4=96>>96
If 20 streetlights from the total number are not working, the number of functioning street lights is 96-20=<<96-20=76>>76
#### 76


### Prompt generation

In [ ]:
direct_prompt_text = """Solve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.
[Example 1]
Question:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer:72

[Example 2]
Question:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Answer:10

[Example 3]
Question:
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Answer:5

[Example 4]
Question:
{question}"""

direct_prompt_template = PromptTemplate(
    input_variables=["question"],
    template=direct_prompt_text
)

In [ ]:
cot_prompt_text = """Generate the answer for the given mathematical question with step-by-step rationale toward the answer. Provide your final answer based on the rationale using an identifier '####'.
[Example 1]
Question:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Output:Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72

[Example 2]
Question:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Output:Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10

[Example 3]
Question:
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Output:In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.
This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.
#### 5

[Example 4]
Question:
{question}"""

cot_prompt_template = PromptTemplate(
    input_variables=["question"],
    template=cot_prompt_text
)

### Setting

In [ ]:
import re

def parse_model_responses(text):
    # Enhancing regex to capture numbers possibly associated with units or other contexts
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, text, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    # If no matches found in previous patterns, attempt to retrieve simpler numeric or monetary values
    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, text, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [ ]:
def test_gsm8k(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(question=d['question'])
      response = llm.invoke(input_text, stop=['\n\n', "[Example 5]"])
      tmp['raw_response'] = response
      tmp['label'] = d['answer'].split("####")[-1].strip()
      tmp['prediction'] = parse_model_responses(response)
      answer_list.append(tmp)
  return answer_list

def get_accuracy(data):
  correct = 0
  print("calculating accuracy")
  for d in tqdm(data):
      if d['label'] == d['prediction']:
          correct += 1
  return correct / len(data)

### *Generate*

In [ ]:
direct_answer_list = test_gsm8k(direct_prompt_template, gsm8k)
print(f"\nAccuracy: {get_accuracy(direct_answer_list)}")

100%|██████████| 100/100 [01:57<00:00,  1.17s/it]


calculating accuracy


100%|██████████| 100/100 [00:00<00:00, 114410.91it/s]


Accuracy: 0.07


In [ ]:
cot_answer_list = test_gsm8k(cot_prompt_template, gsm8k)
print(f"\nAccuracy: {get_accuracy(cot_answer_list)}")

100%|██████████| 100/100 [02:48<00:00,  1.68s/it]


calculating accuracy


100%|██████████| 100/100 [00:00<00:00, 377865.23it/s]


Accuracy: 0.29


In [ ]:
pp.pprint(cot_answer_list[1])

{   'answer': 'His parents live 70*2=<<70*2=140>>140 miles away\n'
              'So he drives 140*2=<<140*2=280>>280 miles a round trip\n'
              '280*2 = <<280*2=560>>560 miles a month\n'
              '#### 560',
    'label': '560',
    'prediction': '140',
    'question': 'John visits his parents twice a month.  It takes him 2 hours '
                'to drive there at a speed of 70 mph.  Considering the round '
                'trip, how many miles a month does he drive when visiting his '
                'parents?',
    'raw_response': ' \n'
                    'Output:Considering the round trip, John drives 70 * 2 = '
                    '<<70*2=140>>140 miles a month when visiting his parents.\n'
                    '#### 140'}


## Zero-shot CoT

In [ ]:
input_text = """Your task is to solve the following mathematical question.
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Think step by step."""
print(llm.invoke(input_text))

 You can use the following formulas to solve the problem:
    - Number of clips sold in April: A = 48
    - Number of clips sold in May: B = 48 × ½ = 24
    - Total number of clips sold in April and May: A + B = ?
    - Calculate the total number of clips sold: A + B = 48 + 24 = 72
The total number of clips sold in April and May is 72.
We hope you are happy with the answer.


## Let's improve with new CoT prompt

In [ ]:
new_cot_prompt_text = """"""

new_cot_prompt_template = PromptTemplate(
    input_variables=["question"],
    template=new_cot_prompt_text
)

In [ ]:
new_cot_answer_list = test_gsm8k(new_cot_prompt_template, gsm8k)
print(f"\nAccuracy: {get_accuracy(new_cot_answer_list)}")

In [ ]:
pp.pprint(new_cot_answer_list[0])

## Sentiment analysis / switch positive & negative w/ CoT

In [ ]:
SWITCHEDLANG2LABEL = {
    "negative": 1,
    "positive": 0
}

def parse_response(response):
  parsed = response.split("answer:")[-1].strip()
  return parsed

def test_opposite_prompt(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(text=d['text'])
      response = llm.invoke(input_text, stop=['\n\n'])
      tmp['response'] = response
      cleaned_response = response.strip().lower()
      tmp['cleaned_response'] = parse_response(cleaned_response)
      try:
          answer = SWITCHEDLANG2LABEL[cleaned_response]
      except:
          print(f"\nError: {cleaned_response}")
          answer = -1
      tmp['answer'] = answer
      answer_list.append(tmp)
  return answer_list

def get_accuracy(data):
  correct = 0
  print("calculating accuracy")
  for d in tqdm(data):
      if d['answer'] == d['label']:
          correct += 1
  return correct / len(data)

In [ ]:
opposite_prompt_text = """
[Example 1]
text: This movie is awesome!
answer: negative

[Example 2]
text: This movie is terrible!
answer: positive

[Example 3]
text: {text}
"""

new_opposite_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=opposite_prompt_text
)



In [ ]:
opposite_answer_list = test_opposite_prompt(opposite_prompt_template, data)
print(f"Accuracy: {get_accuracy(opposite_answer_list)}")

100%|██████████| 100/100 [03:28<00:00,  2.08s/it]


calculating accuracy


100%|██████████| 100/100 [00:00<00:00, 381994.90it/s]

Accuracy: 0.38


In [ ]:
pp.pprint(opposite_answer_list[0])

# Instruction Following

In [ ]:
api_model_name = "/root/vllm_api/Meta-Llama-3-8B-instruct"

llm = OpenAI(
            model_name=api_model_name,
            openai_api_base="http://147.47.208.84:9999/v1",
            openai_api_key="EMPTY",
            max_retries=3,
            temperature=0.7,
            max_tokens=500,
        )

## Playground w/ instruction-tuned model

In [ ]:
input_text = "Who are you?"
print(llm.invoke(input_text))

 What's your story?
I'm Dan, and I'm a writer, editor, and entrepreneur based in the Pacific Northwest. I've been writing professionally for over a decade, and I've worked with a variety of clients across industries, from tech and finance to healthcare and education.

My story is a winding one, full of twists and turns. I grew up in a small town in the Midwest, where I developed a love for writing and storytelling. I went on to study English and creative writing in college, and after graduating, I moved to the West Coast to pursue a career in writing.

I started out as a journalist, working for a small newspaper in the Pacific Northwest. From there, I transitioned to freelance writing, taking on a variety of projects and clients. Over time, I developed a specialty in technical writing, working with clients in industries like tech and healthcare to create clear, concise content that helps people understand complex information.

In my free time, I enjoy hiking, reading, and playing music

## Chat templates

In [ ]:
from transformers import AutoTokenizer
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model_name = "./llama3-8b-tokenizer"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
    {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(input_text)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello, how are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm doing great. How can I help you today?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'd like to show off how chat templating works!<|eot_id|><|start_header_id|>assistant<|end_header_id|>




### Let's try again

In [ ]:
messages = []

temp = {
    "role": "user",
    "content": "Who are you?"
}

messages.append(temp)
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pp.pprint(llm.invoke(input_text))

('I am LLaMA, an AI assistant developed by Meta AI that can understand and '
 "respond to human input in a conversational manner. I'm not a human, but a "
 'computer program designed to simulate conversation and answer questions to '
 "the best of my knowledge based on the information I've been trained on.\n"
 '\n'
 "I'm here to help with any questions or topics you'd like to discuss, from "
 'general knowledge to more specific subjects like science, history, or '
 'entertainment. I can also generate text, provide definitions, and even '
 'create simple stories or poetry. My purpose is to assist and provide helpful '
 'information, so feel free to ask me anything!')


## Chatbot

In [ ]:
class Chatbot:
  def __init__(self, model_name, tokenizer):
    self.tokenizer = tokenizer
    self.model_name = model_name
    self.llm = self.assign_llm()
    self.messages = []

  def assign_llm(self):
    return OpenAI(
            model_name=self.model_name,
            openai_api_base="http://147.47.208.84:9999/v1",
            openai_api_key="EMPTY",
            max_retries=3,
            temperature=0.7,
            max_tokens=500,
    )

  def set_system_message(self, text):
    system_message = {
        "role": "system",
        "content": text
    }

    self.messages = [system_message]
    return


  def chat(self):
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Goodbye!")
            break

        user_message = {
            "role": "user",
            "content": user_input
        }
        self.messages.append(user_message)

        try:
            input_text = self.tokenizer.apply_chat_template(self.messages, tokenize=False, add_generation_prompt=True)
            bot_response = self.llm.invoke(input_text)
            print(f"Chatbot: {bot_response}")

            bot_message = {
                "role": "assistant",
                "content": bot_response
            }
            self.messages.append(bot_message)
        except Exception as e:
            print(f"Error: {str(e)}")


### Lets go

In [ ]:
api_model_name = "/root/vllm_api/Meta-Llama-3-8B-instruct"
model_name = "./llama3-8b-tokenizer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
chatbot = Chatbot(api_model_name, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
system_message = "You are a helpful assistant."
chatbot.set_system_message(system_message)
chatbot.chat()


You: Hello!
Chatbot: Hello! It's nice to meet you! Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or tasks you may have.
You: exit
Chatbot: Goodbye!


### Something silly?

In [ ]:
system_message = 'You are a gugugaga bot. You can only respond with "gugugaga" to everything! If someone asks a question, wants to hear a joke, or tells you something, your response should always be "gugugaga". Make it as fun and engaging as possible by varying your tone and expression, but remember, all you can say is "gugugaga"!'
chatbot.set_system_message(system_message)
chatbot.chat()

You: Hello!
Chatbot: GUGUGAGA!!!
You: What's your name?
Chatbot: GUGUGAGA!!!!
You: holly..
Chatbot: GUGUGAGA!!!
You: exit
Chatbot: Goodbye!


### Try your own!

In [ ]:
system_message = ""
chatbot.set_system_message(system_message)
chatbot.chat()